# 第5章 彩色图像处理

In [1]:
#导入本章示例所用到的包,使用本文档中示例,先运行一次本段代码
import numpy as np
import cv2 as cv
from skimage import color,io,util,exposure,filters
from PIL import Image,ImageEnhance
from scipy import ndimage
from scipy.fft import fft2,ifft2,fftshift,ifftshift
import matplotlib.pyplot as plt
%matplotlib inline

## 5.3 颜色模型
- 颜色模型规定了颜色度量与表达方式的标准。
- 由于颜色模型中每一种颜色一般需要三个与原色相关的基本量来描述，所以常把颜色模型看作是一个多维坐标系统，又称颜色空间（Color space），空间中每一个点都代表一种可能的颜色。
- RGB颜色模型具有设备相关性，真彩色图像通常来源于彩色摄像机、照相机或扫描仪，这些成像装置含有对红、绿和蓝光敏感的传感器，将接收到的红、绿和蓝光能量转换为与其成线性关系的红、绿、蓝分量信号，因此被称为线性RGB值，它是各种颜色模型的基础。由于选用不同的三原色和参考基准白色，出现了多种RGB色度系统，如CIE 1931-RGB、NTSC RGB、sRGB、Adobe RGB等。
- 除了各种RGB颜色模型外，在色度学的研究中，基于不同应用目的建立了多种颜色模型，如CIE 1931-XYZ、CIE 1976-L*a*b*、HSI、HSV、YIQ（NTSC）、YUV、YCbCr、CMY等，本节将介绍几种常用的颜色模型及其之间的转换方法。

## 常用颜色空间转换函数
- OpenCV
    - dst =cv.cvtColor(src, code[, dst[, dstCn]])
    - In case of linear transformations, the range does not matter. But in case of a non-linear transformation, an input RGB image should be normalized to the proper value range to get the correct results.
- Scikit image
    - skimage.color.convert_colorspace(arr, fromspace, tospace)imgBGR = 
    - skimage.color.rgb2hsv(rgb)
    - skimage.color.hsv2rgb(hsv)
    - skimage.color.rgb2ycbcr(rgb)
    - skimage.color.ycbcr2rgb(ycbcr)
    - skimage.color.rgb2gray(rgb)

## 5.3.3 面向视觉感知的颜色模型HSI、HSV
- 面向视觉感知的颜色模型较多，如
    - HSI（Hue，Saturation，Intensity）
    - HSV（Hue，Saturation，Value）
    - HSB（Hue，Saturation，Brightness）
    - HLS或HSL（Hue，Lightness，Saturation）
- 这些颜色模型都是非线性的，用接近人类颜色视觉感知特点的分量来描述颜色的属性。其中HLS或HSL与HSI等同，在OpenCV中称为HLS。

## 5.3.4 视频颜色模型YCbCr
- 国际无线电咨询委员会CCIR (International Radio Consultative Committee)标准601（ITU-R BT.601-5）制订了YCbCr颜色模型，其中Y代表亮度分量，Cb和Cr分别正比于色差（B-Y）和（R-Y），称为色度分量，二者共同描述一种颜色的色调和饱和度属性。
- YCbCr颜色模型是在计算机系统中应用最多的成员，其应用领域很广泛，如JPEG、MPEG均采用此格式。
- YCbCr颜色空间与RGB颜色空间的转换公式有多个版本，下面给出其中的两个。

### OpenCV 颜色空间转换--RGB颜色空间到HLS、HSV、YCbCr

In [ ]:
#OpenCV 颜色空间转换--RGB到HLS、HSV、YCbCr

#RGB值应归一化到区间[0,1]，转换后得到的S和L、V已归一化到区间[0,1]，
#色调 H具有角度的量纲，在区间[0,360]内取值，可除以360将其归一化到区间[0,1]。

#读入一幅 RGB真彩色图像，注意图像数组色序为BGR
img = cv.imread('./imagedata/fruits.jpg',cv.IMREAD_COLOR)

#转换到HLS（HSI）颜色空间
imghls = cv.cvtColor(img.astype(np.float32)/255, cv.COLOR_BGR2HLS)
#转换到 HSV颜色空间
imghsv = cv.cvtColor(img.astype(np.float32)/255, cv.COLOR_BGR2HSV)
#转换到 YCbCr颜色空间
#RGB在[0,255]之间取值，转换后的YCbCr分量在[0,255]间取值
#注意得到的分量存储顺序为 Y、Cr、Cb 
imgycbcr = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)

#由HSV颜色空间转换到RGB空间，注意图像数组色序为RGB
imgrgb = cv.cvtColor(imghsv, cv.COLOR_HSV2RGB)
#由YCbCr颜色空间转换到RGB空间，注意图像数组色序为BGR
imgBGR = cv.cvtColor(imgycbcr, cv.COLOR_YCrCb2BGR)

#显示结果
plt.figure(figsize=(16,9))
plt.gray()

#原RGB图像
plt.subplot(3,4,1); plt.imshow(img[:,:,::-1])
plt.title('Input Image, RGB Color Space') 
plt.axis('off')

#HLS颜色空间
#H分量
plt.subplot(3,4,2); plt.imshow(imghls[:,:,0],vmin=0,vmax=360)
plt.title('H - HLS') 
plt.axis('off')
#S分量
plt.subplot(3,4,3); plt.imshow(imghls[:,:,2],vmin=0,vmax=1)
plt.title('S - HLS') 
plt.axis('off')
#L分量
plt.subplot(3,4,4); plt.imshow(imghls[:,:,1],vmin=0,vmax=1)
plt.title('L - HLS') 
plt.axis('off')

#HSV颜色空间
#由HSV转换得到的RGB图像
plt.subplot(3,4,5); plt.imshow(imgrgb)
plt.title('Converted from HSV to RGB Color Space') 
plt.axis('off')
#H分量
plt.subplot(3,4,6); plt.imshow(imghsv[:,:,0],vmin=0,vmax=360)
plt.title('H - HSV') 
plt.axis('off')
#S分量
plt.subplot(3,4,7); plt.imshow(imghsv[:,:,1],vmin=0,vmax=1)
plt.title('S - HSV') 
plt.axis('off')
#V分量
plt.subplot(3,4,8); plt.imshow(imghsv[:,:,2],vmin=0,vmax=1)
plt.title('V - HSV') 
plt.axis('off')

#YCbCr颜色空间
#由YCbCr转换得到的RGB图像
plt.subplot(3,4,9); plt.imshow(imgBGR[:,:,::-1])
plt.title('Converted from YCbCr to RGB Color Space') 
plt.axis('off')
#Cb分量
plt.subplot(3,4,10); plt.imshow(imgycbcr[:,:,2],vmin=0,vmax=255)
plt.title('Cb - YCbCr') 
plt.axis('off')
#Cr分量
plt.subplot(3,4,11); plt.imshow(imgycbcr[:,:,1],vmin=0,vmax=255)
plt.title('Cr - YCbCr') 
plt.axis('off')
#Y分量
plt.subplot(3,4,12); plt.imshow(imgycbcr[:,:,0],vmin=0,vmax=255)
plt.title('Y - YCbCr') 
plt.axis('off')

plt.show()

### Scikit-image 颜色空间转换--RGB到HSV、YCbCr

In [ ]:
#Scikit-image 颜色空间转换--RGB到HSV、YCbCr
#转换后得到的H、S、V分量已归一化到区间[0,1]
#读入一幅RGB彩色图像
img = io.imread('./imagedata/fruits.jpg')

#将图像颜色空间由RGB转换到HSV
imghsv = color.convert_colorspace(img, 'RGB', 'HSV')

#再由HSV转换到RGB颜色空间
imgrgb = color.hsv2rgb(imghsv)
#将RGB图像数据类型转换为uint8
imgrgb = util.img_as_ubyte(imgrgb)

#将图像颜色空间由RGB转换到YCbCr
#RGB在[0,255]之间取值，转换后的Y分量在[16,235]间取值，Cb和Cr分量在[16,240]之间取值。
imgycbcr = color.rgb2ycbcr(img)

#显示结果
plt.figure(figsize=(16,7))
plt.gray()

#原RGB图像
plt.subplot(2,4,1); plt.imshow(img)
plt.title('Input Image, RGB Color Space') 
plt.axis('off')

#由HSV转换得到的RGB图像
plt.subplot(2,4,5); plt.imshow(imgrgb)
plt.title('Converted from HSV to RGB Color Space') 
plt.axis('off')

#HSV颜色空间
#H分量
plt.subplot(2,4,2); plt.imshow(imghsv[:,:,0],vmin=0,vmax=1)
plt.title('H') 
plt.axis('off')
#S分量
plt.subplot(2,4,3); plt.imshow(imghsv[:,:,1],vmin=0,vmax=1)
plt.title('S') 
plt.axis('off')
#V分量
plt.subplot(2,4,4); plt.imshow(imghsv[:,:,2],vmin=0,vmax=1)
plt.title('V') 
plt.axis('off')

#YCbCr颜色空间
#Cb分量
plt.subplot(2,4,6); plt.imshow(imgycbcr[:,:,1],vmin=16,vmax=240)
plt.title('Cb') 
plt.axis('off')
#Cr分量
plt.subplot(2,4,7); plt.imshow(imgycbcr[:,:,2],vmin=16,vmax=240)
plt.title('Cr') 
plt.axis('off')
#Y分量
plt.subplot(2,4,8); plt.imshow(imgycbcr[:,:,0],vmin=16,vmax=235)
plt.title('Y') 
plt.axis('off')

plt.show()

### RGB彩色图像转换为灰度图像

In [ ]:
# RGB彩色图像转换为灰度图像

#读入一幅RGB彩色图像
img = io.imread('./imagedata/astronaut.png')

#Scikit-image:将RGB彩色图像转换为灰度图像
img_gray = color.rgb2gray(img)
#将图像数据类型转换为uint8
imggray1 = util.img_as_ubyte(img_gray)

#OpenCV:将RGB彩色图像转换为灰度图像
imggray2 = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

#显示结果
plt.figure(figsize=(12,8))

#原RGB彩色图像
plt.subplot(1,3,1); plt.imshow(img)
plt.title('Input Image, RGB Color Space') 
plt.axis('off')
#Scikit-image得到的灰度图像
plt.subplot(1,3,2); plt.imshow(imggray1,vmin=0,vmax=255,cmap='gray')
plt.title('Gray image by Scikit-image') 
plt.axis('off')
#OpenCV得到的灰度图像
plt.subplot(1,3,3); plt.imshow(imggray2,vmin=0,vmax=255,cmap='gray')
plt.title('Gray image by OpenCV') 
plt.axis('off')

plt.show()

### 5.4.1 RGB真彩色图像
- RGB真彩色图像的颜色，用红R、绿G、蓝B三个分量来描述，各分量通常采用8位无符号整数进行数字化，共24位组合，能够产生16777216种不同的颜色。
- 像素的颜色分量（color component），又称为颜色通道（color channel）、颜色平面（color plane）。
- Python语言中，Matplotlib、Scikit-image、SciPy等用一个NumPy ndarray型三维数组来保存RGB真彩色图像数据。第1维是数组的行序，M是数组的行数，对应图像的高度；第2维是数组的列序，N是数组的列数，对应图像的宽度；第3维称为色序。这样就可以用数组的行序、列序和色序3个下标，来读、写像素的三个分量值。改写图像数组元素值，就可以改变该像素的颜色。
- 需注意的是，OpenCV函数cv.imread()以彩色方式读取图像时，返回NumPy三维数组的颜色通道顺序为BGR。

In [ ]:
#RGB真彩色图像及其分量

#读入一幅RGB彩色图像
img = io.imread('./imagedata/fruits.jpg')

#显示图像
plt.figure(figsize=(16,6))
plt.gray()

#RGB彩色图像
plt.subplot(2,4,1); plt.imshow(img)
plt.title('Original RGB Color Space') 
plt.axis('off')

#以灰度图像方式显示RGB各分量
#R分量
plt.subplot(2,4,2); plt.imshow(img[:,:,0],vmin=0,vmax=255)
plt.title('R') 
plt.axis('off')
#G分量
plt.subplot(2,4,3); plt.imshow(img[:,:,1],vmin=0,vmax=255)
plt.title('G') 
plt.axis('off')
#B分量
plt.subplot(2,4,4); plt.imshow(img[:,:,2],vmin=0,vmax=255)
plt.title('B') 
plt.axis('off')

#以彩色图像方式显示RGB分量
img_r = img.copy()
img_r[:,:,1:3] = 0
img_g = img.copy()
img_g[:,:,0] = 0; img_g[:,:,2] = 0
img_b = img.copy()
img_b[:,:,0:2] = 0
#R分量
plt.subplot(2,4,6); plt.imshow(img_r)
plt.title('R') 
plt.axis('off')
#G分量
plt.subplot(2,4,7); plt.imshow(img_g)
plt.title('G') 
plt.axis('off')
#B分量
plt.subplot(2,4,8); plt.imshow(img_b)
plt.title('B') 
plt.axis('off')

plt.show()

## 5.5 彩色图像的点处理
- 类似于灰度变换，彩色图像的点处理按照指定的函数关系，将输入彩色图像像素的颜色分量映射为输出图像对应像素的颜色分量，这种映射完全取决于输入图像中各像素颜色分量和所选择的变换函数，而与邻域像素无关，又称彩色变换。
- 不同于灰度变换，彩色图像的点处理需要处理每个像素的多个颜色分量，这些颜色分量可以是RGB，或HSI、HSV、YCbCr、CMYK等其它颜色空间。

### 5.5.1 彩色图像的饱和度调整
- 对于RGB真彩色图像，降低或增强其颜色饱和度，可以直接在RGB颜色空间实现，通过在每个像素的颜色点（R,G,B）与其对应灰度点（Y,Y,Y）之间进行线性插值来获得。
- 图像饱和度的调整也可以在HSI或HSV颜色空间中进行，先把RGB彩色图像转换到HSI或HSV颜色空间，调整图像饱和度分量S的大小，最后再转换到RGB颜色空间。

In [ ]:
#RGB彩色图像的饱和度调整
#读入一幅RGB彩色图像
img = io.imread('./imagedata/fruits.jpg')

#定义饱和度控制因子的大小
#sc=0,消色,得到无彩图像;#sc=1,不改变饱和度;sc>1,加大饱和度;sc<1,降低饱和度。 
sc = 2

#在RGB颜色空间调整饱和度
imggray = np.mean(img.astype(float),2)  #转换为灰度图像
imggray3 = np.dstack((imggray,imggray,imggray))  #构造三维数组
imgse1 = imggray3 + sc * (img.astype(float) - imggray3)
#对计算结果作饱和处理,限制在[0,255]之间
imgse1 = np.clip(imgse1,0,255)
#将RGB图像数据类型转换为uint8
imgse1 = np.uint8(imgse1)

#在HSV颜色空间调整饱和度
imghsv = color.rgb2hsv(img)
imghsv[:,:,1] = sc * imghsv[:,:,1]
#对计算结果作饱和处理,限制在[0,1]之间
imghsv[:,:,1] = np.clip(imghsv[:,:,1],0,1)
#将图像转换到RGB颜色空间
imgse2 = color.hsv2rgb(imghsv)
#将RGB图像数据类型转换为uint8
imgse2 = util.img_as_ubyte(imgse2)

#在HLS(HSI)颜色空间调整饱和度
imghls = cv.cvtColor(img.astype(np.float32)/255, cv.COLOR_RGB2HLS)
imghls[:,:,2] = sc * imghls[:,:,2]
#对计算结果作饱和处理,限制在[0,1]之间
imghls[:,:,2] = np.clip(imghls[:,:,2],0,1)
#将图像转换到RGB颜色空间
imgse3  = cv.cvtColor(imghls, cv.COLOR_HLS2RGB)
#将RGB图像数据类型转换为uint8
imgse3  = util.img_as_ubyte(imgse3)

#显示结果
plt.figure(figsize=(16,8))
#原图像
plt.subplot(1,4,1); plt.imshow(img)
plt.title('Original image') 
plt.axis('off')
#在RGB颜色空间调整饱和度
plt.subplot(1,4,2); plt.imshow(imgse1)
plt.title('Saturation adjusted in RGB') 
plt.axis('off')
#在HSV颜色空间调整饱和度
plt.subplot(1,4,3); plt.imshow(imgse2)
plt.title('Saturation adjusted in HSV') 
plt.axis('off')
#在HLS(HSI)颜色空间调整饱和度
plt.subplot(1,4,4); plt.imshow(imgse3)
plt.title('Saturation adjusted in HLS(HSI)') 
plt.axis('off')

plt.tight_layout()
plt.show()

### 5.5.2 亮度和对比度调整

### 示例：Scikit-image具有饱和处理的线性灰度变换函数增强彩色图像

In [ ]:
#Scikit-image: 增强彩色图像的亮度和对比度
#读入RGB彩色图像
img1 = io.imread('./imagedata/chalk_bright.tif')
img2 = io.imread('./imagedata/flower_flat.tif')
img3 = io.imread('./imagedata/stream_dark.tif')

#采用百分位数（percentile）来选择rlow和rhigh
rlow, rhigh = np.percentile(img1,(5, 95))
#灰度值拉伸
img1_rescale = exposure.rescale_intensity(img1, in_range=(rlow, rhigh))

#采用百分位数（percentile）来选择rlow和rhigh
rlow, rhigh = np.percentile(img2,(5, 95))
#灰度值拉伸
img2_rescale = exposure.rescale_intensity(img2, in_range=(rlow, rhigh))

#采用百分位数（percentile）来选择rlow和rhigh
rlow, rhigh = np.percentile(img3,(5, 95))
#灰度值拉伸
img3_rescale = exposure.rescale_intensity(img3, in_range=(rlow, rhigh))

#显示结果
plt.figure(figsize=(12,8))
#原图像chalk_bright
plt.subplot(2,3,1); plt.imshow(img1)
plt.title('Original image,chalk_bright') 
plt.axis('off')
#对比度拉伸结果
plt.subplot(2,3,4); plt.imshow(img1_rescale) 
plt.title('Contrast stretching by percentile ')
plt.axis('off')
#原图像flower_flat
plt.subplot(2,3,2); plt.imshow(img2)
plt.title('Original image,flower_flat') 
plt.axis('off')
#对比度拉伸结果
plt.subplot(2,3,5); plt.imshow(img2_rescale) 
plt.title('Contrast stretching by percentile '), plt.axis('off')
#原图像stream_dark
plt.subplot(2,3,3); plt.imshow(img3)
plt.title('Original image,stream_dark'), plt.axis('off')
#对比度拉伸结果
plt.subplot(2,3,6); plt.imshow(img3_rescale) 
plt.title('Contrast stretching by percentile '), plt.axis('off')

plt.show()
#-----------------------

## 示例：Scikit-image彩色图像的直方图均衡化

In [ ]:
#Scikit-image彩色图像的全局直方图均衡化
#读入RGB彩色图像
imgchalk = io.imread('./imagedata/chalk_bright.tif')
imgflower = io.imread('./imagedata/flower_flat.tif')
imgstream = io.imread('./imagedata/stream_dark.tif')

#对亮度分量直方图均衡化
#chalk_bright
imgchalk_hsv = color.rgb2hsv(imgchalk)
imgchalk_hsv[:,:,2] = exposure.equalize_hist(imgchalk_hsv[:,:,2])
imgchalk_heq = color.hsv2rgb(imgchalk_hsv)
imgchalk_heq = util.img_as_ubyte(imgchalk_heq)
#flower_flat
imgflower_hsv = color.rgb2hsv(imgflower)
imgflower_hsv[:,:,2] = exposure.equalize_hist(imgflower_hsv[:,:,2])
imgflower_heq = color.hsv2rgb(imgflower_hsv)
imgflower_heq = util.img_as_ubyte(imgflower_heq)
#stream_dark
imgstream_hsv = color.rgb2hsv(imgstream)
imgstream_hsv[:,:,2] = exposure.equalize_hist(imgstream_hsv[:,:,2])
imgstream_heq = color.hsv2rgb(imgstream_hsv)
imgstream_heq = util.img_as_ubyte(imgstream_heq)

#所有分量视为同一个数据集的全局直方图均衡化
imgc_heq = exposure.equalize_hist(imgchalk)
imgf_heq = exposure.equalize_hist(imgflower)
imgs_heq = exposure.equalize_hist(imgstream)

#显示结果
plt.figure(figsize=(12,12))

#原图像
plt.subplot(3,3,1), plt.imshow(imgchalk)
plt.title('Original chalk image') 
plt.axis('off')
plt.subplot(3,3,2), plt.imshow(imgflower)
plt.title('Original flower image') 
plt.axis('off')
plt.subplot(3,3,3), plt.imshow(imgstream)
plt.title('Original stream image') 
plt.axis('off')
#对亮度分量直方图均衡化结果
plt.subplot(3,3,4), plt.imshow(imgchalk_heq) 
plt.title('GHE_V,chalk')
plt.axis('off')
plt.subplot(3,3,5), plt.imshow(imgflower_heq) 
plt.title('GHE_V,flower')
plt.axis('off')
plt.subplot(3,3,6), plt.imshow(imgstream_heq)
plt.title('GHE_V,stream')
plt.axis('off')
#所有分量视为同一个数据集的全局直方图均衡化结果
plt.subplot(3,3,7), plt.imshow(imgc_heq) 
plt.title('GHE_RGB,chalk')
plt.axis('off')
plt.subplot(3,3,8), plt.imshow(imgf_heq) 
plt.title('GHE_RGB,flower')
plt.axis('off')
plt.subplot(3,3,9), plt.imshow(imgs_heq) 
plt.title('GHE_RGB,stream ')
plt.axis('off')

plt.show()

### OpenCV 彩色图像的直方图均衡化，通过对亮度V分量进行全局直方图均衡化来实现

In [ ]:
#OpenCV彩色图像的直方图均衡化

#读入一幅彩色图像
img = cv.imread('./imagedata/temple.jpg')

#将图像颜色空间转换到HSV颜色空间
imghsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
imghsv_old = imghsv.copy()

#对L通道分量进行全局直方图均衡化
imgv_eq1 = cv.equalizeHist(imghsv[:,:,2])

#对V通道分量采用CLAHE直方图均衡化
clahe = cv.createCLAHE(clipLimit= 2.0,tileGridSize=(8,8))
imgv_eq2 = clahe.apply(imghsv[:,:,2])

imghsv[:,:,2] = imgv_eq1  
#将颜色空间转换为BGR
imgeq1 = cv.cvtColor(imghsv, cv.COLOR_HSV2BGR)
imghsv[:,:,2] = imgv_eq2  
#将颜色空间转换为BGR
imgeq2 = cv.cvtColor(imghsv, cv.COLOR_HSV2BGR)

#显示结果
plt.figure(figsize=(16,10))
#原彩色图像
plt.subplot(3,3,1); plt.imshow(img[:,:,::-1])
plt.title('temple,Original') 
plt.axis('off')
#全局直方图均衡化结果
plt.subplot(3,3,2); plt.imshow(imgeq1[:,:,::-1])
plt.title('GHE-Global Histogram Equalization') 
plt.axis('off')

#全CLAHE直方图均衡化结果
plt.subplot(3,3,3); plt.imshow(imgeq2[:,:,::-1])
plt.title('CLAHE') 
plt.axis('off')

#计算彩色图像RGB颜色通道的直方图，并以叠加方式显示
#原图像
plt.subplot(3,3,4)
plt.hist(img[:,:,0].ravel(), bins=256,density=1, color='b')
plt.hist(img[:,:,1].ravel(), bins=256,density=1, color='g')
plt.hist(img[:,:,2].ravel(), bins=256,density=1, color='r')
plt.title('Original color histogram') 

#全局直方图均衡化GHE图像
plt.subplot(3,3,5)
plt.hist(imgeq1[:,:,0].ravel(), bins=256,density=1, color='b')
plt.hist(imgeq1[:,:,1].ravel(), bins=256,density=1, color='g')
plt.hist(imgeq1[:,:,2].ravel(), bins=256,density=1, color='r')
plt.title('GHE color histogram') 

#采用CLAHE直方图均衡化图像
plt.subplot(3,3,6)
plt.hist(imgeq2[:,:,0].ravel(), bins=256,density=1, color='b')
plt.hist(imgeq2[:,:,1].ravel(), bins=256,density=1, color='g')
plt.hist(imgeq2[:,:,2].ravel(), bins=256,density=1, color='r')
plt.title('CLAHE color histogram') 

#显示均衡化前后图像V通道的直方图
#原图像
plt.subplot(3,3,7)
plt.hist(imghsv_old[:,:,2].ravel(), bins=256,density=1, color='black')
plt.title('Original V histogram') 
#全局直方图均衡化GHE图像
plt.subplot(3,3,8)
plt.hist(imgv_eq1.ravel(), bins=256,density=1, color='black')
plt.title('GHE V histogram')
#CLAHE均衡化GHE图像
plt.subplot(3,3,9)
plt.hist(imgv_eq2.ravel(), bins=256,density=1, color='black')
plt.title('CLAHE V histogram')

plt.show()

## 示例： 连续单独或同时调整图像的亮度和对比度灰度
- 先运行以下自定义函数Cell

In [21]:
#自定义连续单独或同时调整图像的亮度和对比度
def adjust_bright_contrast(image,b,c):
    """
    输入参数: image - 图像数组，灰度或彩色图像，数据类型uint8;
                 b - 在区间[-1, 1]内取值.b<0,降低亮度;b>0,提高亮度;
                 c - 在区间[-1, 1]内取值.c<0,降低对比度;c>0,提高对比度;
    返回参数：img_out- 灰度变换结果，数组，数据类型uint8;
    """
    k = np.tan((45 + 44 * c)* np.pi/180)
    img_out = (image.astype(float) - 127.5*(1 - b)) * k + 127.5 * (1 + b)
    
    #将输出图像数据类型转换为uint8型
    img_out= np.clip(img_out,0,255).astype(np.uint8)    
    #返回结果
    return img_out
#----------------------------------------------------

In [ ]:
#使用自定义函数adjust_bright_contrast调整图像的亮度和对比度

#读入RGB彩色图像
img1 = io.imread('./imagedata/chalk_bright.tif')
img2 = io.imread('./imagedata/flower_flat.tif')
img3 = io.imread('./imagedata/stream_dark.tif')

#调整图像亮度和对比度
img1_out = adjust_bright_contrast(img1,-0.2, 0.3)
img2_out = adjust_bright_contrast(img2,-0.1, 0.2)
img3_out = adjust_bright_contrast(img3,0.4,0.5)

#显示结果
plt.figure(figsize=(12,8))

#原图像
plt.subplot(2,3,1); plt.imshow(img1)
plt.title('chalk_bright,Original') 
plt.axis('off')
plt.subplot(2,3,2); plt.imshow(img2)
plt.title('flower_flat,Original') 
plt.axis('off')
plt.subplot(2,3,3), plt.imshow(img3)
plt.title('stream_dark,Original') 
plt.axis('off')

#亮度和对比度调整后的图像
plt.subplot(2,3,4); plt.imshow(img1_out)
plt.title('chalk_bright,b=-0.8,c=0.3')
plt.axis('off')
plt.subplot(2,3,5); plt.imshow(img2_out)
plt.title('flower_flat,b=-0.1,c=0.2')
plt.axis('off')
plt.subplot(2,3,6); plt.imshow(img3_out)
plt.title('stream_dark,b=0.4,c=0.5')
plt.axis('off')

plt.show()
#-----------------

### Pillow彩色图像增强

In [ ]:
#Pillow彩色图像增强

#读入一幅RGB彩色图像
img = Image.open('./imagedata/fruits.jpg')

#调整图像颜色饱和度
#创建一个颜色增强类对象
enh_color = ImageEnhance.Color(img)
#调用对象方法函数增强图像颜色饱和度
img_c = enh_color.enhance(1.5)

#调整图像对比度
#创建一个对比度增强类对象
enh_contrast = ImageEnhance.Contrast(img)
#调用对象方法函数增强图像对比度
img_con = enh_contrast.enhance(1.2)

#调整图像亮度
#创建一个亮度增强类对象
enh_brightness = ImageEnhance.Brightness(img)
#调用对象方法函数增强图像亮度
img_b = enh_brightness.enhance(1.2)

#调整图像锐度
#创建一个锐度增强类对象
enh_sharpness = ImageEnhance.Sharpness(img)
#调用对象方法函数增强图像锐度
img_s = enh_sharpness.enhance(2)

#显示结果
plt.figure(figsize=(15,6))

#原图像
plt.subplot(2,4,1); plt.imshow(img)
plt.title('Original image') 
plt.axis('off')
#饱和度调整结果
plt.subplot(2,4,2); plt.imshow(img_c)
plt.title('Saturation enhanced') 
plt.axis('off')
#对比度调整结果
plt.subplot(2,4,3); plt.imshow(img_con)
plt.title('Contrast enhanced') 
plt.axis('off')
#亮度调整结果
plt.subplot(2,4,4); plt.imshow(img_b)
plt.title('Brightness enhanced') 
plt.axis('off')
#锐度调整结果
plt.subplot(2,4,5); plt.imshow(img_s)
plt.title('Sharpness enhanced') 
plt.axis('off')

plt.tight_layout()
plt.show()

## 5.5.3 彩色图像色调的调整

- “色相”是色调hue的别称。如果对图像中每一个像素的色调分量H加一个正（或负）的常数值（角度值），将会使每个像素的颜色在色相环上沿逆时针（或顺时针）方向移动，当这个常数较小时，一般会导致图像的色调变成“暖”或“冷”色调，而当这个数比较大时，则有可能会使图像的色彩发生较显著的变化。
- 在调整图像色调时，应将色调分量H的值变换到区间[0,360]内，并以360为模取余数，然后再归一化到区间[0,1]。

### 示例：通过加减H分量改变彩色图像的色调

In [ ]:
#通过增大或减小 H分量改变彩色图像的色调

#读入RGB彩色图像
img = io.imread('./imagedata/colorpencil.png')

#将图像转换到HSV颜色空间
imghsv = color.rgb2hsv(img)

#设定色调增加的度数
hdeg = 200
#先将色调分量H从[0,1]映射到[0,360],加上一个度数hdeg后，360取模再归一化
imghsv[:,:,0] = np.mod((imghsv[:,:,0]*360 + hdeg),360)/360
#将图像转换到RGB颜色空间
imgrgb = color.hsv2rgb(imghsv)
#将图像数据类型转换为uint8
imgrgb = util.img_as_ubyte(imgrgb)

#显示结果
plt.figure(figsize=(12,6))

#原彩色图像
plt.subplot(1,2,1); plt.imshow(img)
plt.title('Original image') 
plt.axis('off')
#色调调整结果
plt.subplot(1,2,2); plt.imshow(imgrgb) 
plt.title('Hue adjusted ')
plt.axis('off')

plt.show()

### 示例：怀旧滤镜
- 通过色调调整， 让一幅照片呈现褪色泛黄的老旧照片的视觉， 增强图片的历史韵味。 
- 各类图像编辑软件一般都提供一种称为“ 滤镜” 的图像处理工具， 实现诸如怀旧、 复古、 清新等功能。

In [ ]:
#彩色图像的怀旧滤镜处理
#定义怀旧滤镜变换矩阵
Filter_olddays =np.array([[0.393, 0.769, 0.189],
                          [0.349, 0.686, 0.168],
                          [0.272, 0.534, 0.131]])

#读取一幅RGB彩色图像
img = io.imread('./imagedata/boats.jpg')

#矩阵运算
imgolddays = img.astype(np.float) @ Filter_olddays.T  #矩阵转置
#将图像数据转换为uint8
imgolddays = np.uint8(np.clip(imgolddays,0,255))

#显示结果
plt.figure(figsize=(12,8))

#原图像
plt.subplot(1,2,1); plt.imshow(img)
plt.title('Original image') 
plt.axis('off')
#色调调整结果
plt.subplot(1,2,2); plt.imshow(imgolddays) 
plt.title('Nostalgic filtered result ')
plt.axis('off')

plt.show()

### 示例：Scikit-image 彩色图像直方图匹配 match_histograms

In [ ]:
#Scikit-image 彩色图像直方图匹配 match_histograms

#读取两幅彩色图像
#原图像
img = io.imread('./imagedata/chelsea.png')
#参考图像，指定匹配目标
imgref = io.imread('./imagedata/coffee.png')

#进行直方图匹配
imgmatched = exposure.match_histograms(img,imgref, channel_axis=-1)

#显示结果
plt.figure(figsize=(12,6))

#原彩色图像
plt.subplot(2,3,1); plt.imshow(img)
plt.title('chelsea,original image') 
plt.axis('off')
#参考图像
plt.subplot(2,3,2); plt.imshow(imgref)
plt.title('coffee, reference image') 
plt.axis('off')
#直方图匹配结果
plt.subplot(2,3,3); plt.imshow(imgmatched)
plt.title('histograms matched result') 
plt.axis('off')

#计算彩色图像RGB颜色分量的直方图，并以叠加方式显示
#原图像颜色分量直方图
plt.subplot(2,3,4)
plt.hist(img[:,:,0].ravel(), bins=256, color='r',density=1)
plt.hist(img[:,:,1].ravel(), bins=256, color='g',density=1)
plt.hist(img[:,:,2].ravel(), bins=256, color='b',density=1)
plt.title('chelsea color histogram') 

#参考图像颜色分量直方图
plt.subplot(2,3,5)
plt.hist(imgref[:,:,0].ravel(), bins=256, color='r',density=1)
plt.hist(imgref[:,:,1].ravel(), bins=256, color='g',density=1)
plt.hist(imgref[:,:,2].ravel(), bins=256, color='b',density=1)
plt.title('coffee color histogram') 

#直方图匹配后的图像颜色分量直方图
plt.subplot(2,3,6)
plt.hist(imgmatched[:,:,0].ravel(), bins=256, color='r',density=1)
plt.hist(imgmatched[:,:,1].ravel(), bins=256, color='g',density=1)
plt.hist(imgmatched[:,:,2].ravel(), bins=256, color='b',density=1)
plt.title('matched result color histogram') 

plt.show()
#------------------------------------

## 5.6 彩色图像的滤波处理
- 彩色图像的滤波多在RGB颜色空间中实施。将RGB真彩色图像的每个颜色分量视为一幅灰度图像，然后用同样的滤波模板，使用灰度图像的滤波方法单独处理R、G、B分量图像，再把处理后的三个颜色分量组合起来，就完成了彩色图像的滤波处理。
- 对于索引图像，不能直接应用为灰度图像的滤波方法。应先将索引图像转换为RGB真彩色图像，按照上述方法处理完之后再将其转换为索引图像。

### 5.6.1 彩色图像的平滑与降噪
### 示例：用均值滤波器在空域滤波平滑彩色图像

In [ ]:
#SciPy的ndimage以及OpenCV提供的均值滤波函数

#读入RGB彩色图像
img = io.imread('./imagedata/bigeyemonkey.jpg')

#SciPy,对彩色图RGB颜色通道分别施加9*9均值平滑滤波
imgblur_r = ndimage.uniform_filter(img[:,:,0] ,size = 9)
imgblur_g = ndimage.uniform_filter(img[:,:,1] ,size = 9)
imgblur_b = ndimage.uniform_filter(img[:,:,2] ,size = 9)
#把处理后的RGB三个颜色通道组合起来
imgblur_sc = np.dstack((imgblur_r,imgblur_g,imgblur_b))

#OpenCV,直接将彩色图像数组作为输入
imgblur_cv = cv.blur(img,(9,9))

#显示结果
plt.figure(figsize=(16,8))

#原图像
plt.subplot(1,3,1); plt.imshow(img)
plt.title('Original image') 
plt.axis('off')
#Scipy函数平滑滤波结果
plt.subplot(1,3,2); plt.imshow(imgblur_sc) 
plt.title('Average filter result by SciPy ')
plt.axis('off')
#OpenCV函数平滑滤波结果
plt.subplot(1,3,3); plt.imshow(imgblur_cv) 
plt.title('Average filter result by OpenCV')
plt.axis('off')

plt.show()

### 示例：用高斯低通滤波器在频域平滑彩色图像

In [ ]:
#用高斯低通滤波器在频域平滑彩色图像

#读入RGB彩色图像
img = io.imread('./imagedata/bigeyemonkey.jpg')

#获取图像高/宽大小
M,N = img.shape[0:2]

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N),(0,0)),mode='reflect')

#计算扩展图像的DFT,并中心化(分别对RGB三个颜色分量做DFT)
Fp = fft2(imgex,axes=(0,1))
Fp = fftshift(Fp,axes=(0,1))

#初始化高斯低通滤波器的截止频率
D0 = 50
#计算高斯低通滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HGlpf = np.exp(-Da2/(2*D0**2))
#把高斯低通滤波器串成一个2M*2N*3的三维数组,分别处理RGB颜色通道
HGlpf_3D = np.dstack((HGlpf,HGlpf,HGlpf))

#计算图像DFT与高斯低通滤波器频域传递函数的乘积
Gp = Fp * HGlpf_3D
#去中心化
Gp = ifftshift(Gp,axes=(0,1))
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp,axes=(0,1)))
#把输出图像的数据格式转换为uint8
imgp = np.uint8(np.clip(imgp,0,255))
#截取imgp左上角与原图像大小相等的区域作为输出
imgout = imgp[0:M,0:N]

#显示结果
plt.figure(figsize=(18,8))

#原图像
plt.subplot(1,4,1); plt.imshow(img)
plt.title('Input Image') 
plt.axis('off')
#高斯低通滤波结果
plt.subplot(1,4,2); plt.imshow(imgout)
plt.title('Output Image') 
plt.axis('off')
#高斯低通滤波结果（未裁剪）
plt.subplot(1,4,3); plt.imshow(imgp)
plt.title('Output Image, not clipped') 
plt.axis('off')
#高斯低通滤波器幅度谱
plt.subplot(1,4,4); plt.imshow(np.abs(HGlpf),cmap = 'gray')
plt.title('Gaussian Low Pass Filter Spectrum')
plt.axis('off')

plt.show()

### 示例：用中值滤波器去除RGB彩色图像中的脉冲噪声

In [ ]:
#SciPy,OpenCV的彩色图像中值滤波

#读入RGB彩色图像
img = io.imread('./imagedata/bigeyemonkey.jpg')

#向图像中添加椒盐噪声,密度为0.2
img_noise = util.random_noise(img,mode='s&p',amount=0.2)
img_noise = util.img_as_ubyte(img_noise)

imgresult1 = img.copy()  #复制图像数组，初始化输出数组

#SciPy,分别对RGB颜色通道进行5*5中值滤波
imgresult1[:,:,0] = ndimage.median_filter(img_noise[:,:,0],size=5)
imgresult1[:,:,1] = ndimage.median_filter(img_noise[:,:,1],size=5)
imgresult1[:,:,2] = ndimage.median_filter(img_noise[:,:,2],size=5)

#OpenCV,直接将彩色图像数组作为输入
imgresult2 = cv.medianBlur(img_noise,5)

#显示结果
plt.figure(figsize=(16,8))

#原图像
plt.subplot(1,4,1); plt.imshow(img)
plt.title('Original image') 
plt.axis('off')
#添加椒盐噪声的图像
plt.subplot(1,4,2); plt.imshow(img_noise)
plt.title('added salt & pepper noise') 
plt.axis('off')
#SciPy5*5中值滤波结果
plt.subplot(1,4,3); plt.imshow(imgresult1)
plt.title('median filter result_SciPy') 
plt.axis('off')
#OpenCV5*5中值滤波结果
plt.subplot(1,4,4); plt.imshow(imgresult2)
plt.title('median filter result_OpenCV') 
plt.axis('off')

plt.show()

### 5.6.2 彩色图像的锐化
### 示例：用拉普拉斯算子锐化彩色图像
- 对彩色图像的锐化时，将R、G、B分量看作灰度图像，分别锐化，再组合起来，得到最终锐化结果。

In [ ]:
#采用8-邻域拉普拉斯算子锐化彩色图像

#读入一幅彩色图像
img = io.imread('./imagedata/pears.png')

#构造8-邻域拉普拉斯算子
klap8 = np.array([[1,1,1],
                  [1,-8,1],
                  [1,1,1]])
#强度因子
alpha = 1.5

#采用高斯低通滤波器对原图像滤波,降低噪声
img_smooth = ndimage.gaussian_filter(img,sigma=1,output=np.float64)

#计算RGB各颜色通道8-邻域拉普拉斯算子边缘图像
imgedge_r = ndimage.convolve(img_smooth[:,:,0],klap8,output=np.float64)
imgedge_g = ndimage.convolve(img_smooth[:,:,1],klap8,output=np.float64)
imgedge_b = ndimage.convolve(img_smooth[:,:,2],klap8,output=np.float64)
#把处理后的RGB三个颜色通道边缘组合起来
imgedge = np.dstack((imgedge_r,imgedge_g,imgedge_b))

#将边缘叠加到原图像上
img_sharpen = img.astype(np.float64)- alpha * imgedge
#对结果数据做饱和处理并转换为uint8
img_sharpen= np.uint8(np.clip(img_sharpen,0,255))

#显示结果
plt.figure(figsize=(12,8))
#原图像
plt.subplot(1,2,1); plt.imshow(img)
plt.title('Original') 
plt.axis('off')
#laplace-8锐化结果
plt.subplot(1,2,2); plt.imshow(img_sharpen)
plt.title('Laplace-8 Sharpening') 
plt.axis('off')

plt.tight_layout()
plt.show()

### 示例：钝化掩膜（USM）锐化彩色图像
- 对彩色图像的RGB各个颜色通道单独锐化，可能会导致图像出现颜色扩散现象。如果把图像从RGB转换到HSV、HSI或YCbCr颜色空间，仅对亮度分量进行USM锐化，然后再转换到RGB颜色空间，就可以获得更令人满意的视觉效果。

In [ ]:
#采用Scikit-image函数对彩色图像进行钝化掩膜（USM）锐化

#读入一幅彩色图像
img = io.imread('./imagedata/pears.png')

#将彩色图像转换到HSV颜色空间
img_hsv = color.rgb2hsv(img)
#仅对亮度分量V进行USM锐化
img_hsv[:,:,2]= filters.unsharp_mask(img_hsv[:,:,2], radius=1.5, amount=2.0)
#将彩色图像转换回RGB颜色空间
img_usm = color.hsv2rgb(img_hsv)
#将数据类型转换为uint8
img_usm = util.img_as_ubyte(img_usm)

#显示结果
plt.figure(figsize=(12,8))
#原图像
plt.subplot(1,2,1); plt.imshow(img)
plt.title('Original') 
plt.axis('off')
#USM锐化结果
plt.subplot(1,2,2); plt.imshow(img_usm)
plt.title('USM Sharpening result') 
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#采用Scikit-image函数对彩色图像进行钝化掩膜（USM）锐化
# 新版本调用方法，channel_axis was added in 0.19.

#读入一幅彩色图像
img = io.imread('./imagedata/pears.png')
img_usm = filters.unsharp_mask(img, radius=1.5, amount=2.0, preserve_range=False,channel_axis =2 )
#将数据类型转换为uint8
img_usm = util.img_as_ubyte(img_usm)

#显示结果
plt.figure(figsize=(12,8))
#原图像
plt.subplot(1,2,1); plt.imshow(img)
plt.title('Original') 
plt.axis('off')
#USM锐化结果
plt.subplot(1,2,2); plt.imshow(img_usm)
plt.title('USM Sharpening result') 
plt.axis('off')

plt.tight_layout()
plt.show()

### 示例：采用同态滤波器增强光照不均匀彩色图像


In [ ]:
#同态滤波器(Homomorphic Filter)增强不均匀光照彩色图像
#读入一幅彩色图像
img = io.imread('./imagedata/temple.jpg')
#获取图像高/宽大小
M,N = img.shape[0:2]

#将彩色图像转换到HSV颜色空间
img_hsv = color.rgb2hsv(img)

#仅对亮度分量V进行同态滤波处理
imgV = 255 * img_hsv[:,:,2]
#图像img取对数,得到imgz
imgz = np.log(imgV + 1)

#计算对数图像imgz的DFT并中心化
Zp = fftshift(fft2(imgz,s=(2*M,2*N)))

#初始化同态滤波器参数
rL=0.4    
rH=1.5
#巴特沃斯滤波器的阶次
n = 2    
#滤波器的截止频率
D0 = 1.0 * np.max((2*M,2*N)) 

#计算同态滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da = np.sqrt(Ua**2 + Va**2)
HBlpf = 1/(1+(Da/D0)**(2*n)) 
Homf =  rL * HBlpf + rH * (1 - HBlpf)

#计算同态滤波后的图像频谱,并去中心化
Sp = ifftshift(Zp * Homf)
#DFT反变换,并取实部
imgp = np.real(ifft2(Sp))
#截取imgp左上角M*N的区域,取自然指数,并减1补偿
imgV_enh = np.exp(imgp[0:M,0:N])-1
#把输出图像V分量的数据映射到[0,1]之间
imgV_enh = (imgV_enh - np.min(imgV_enh))/(np.max(imgV_enh) - np.min(imgV_enh))
img_hsv[:,:,2] = imgV_enh
#将彩色图像转换回RGB颜色空间
img_homf = color.hsv2rgb(img_hsv)
#将数据类型转换为uint8
img_homf = util.img_as_ubyte(img_homf)

#显示结果
plt.figure(figsize=(12,8))
#原图像
plt.subplot(1,2,1); plt.imshow(img)
plt.title('Input Image') 
plt.axis('off')
#同态滤波器增强结果
plt.subplot(1,2,2); plt.imshow(img_homf)
plt.title('Homomorphic Filter enhancement result') 
plt.axis('off')

plt.show()
#---------------------------

## 5.7 图像的伪彩色处理
- 人类视觉对微小的灰度变化不敏感，而对彩色的微小差别极为敏感。伪彩色图像处理（pesudo color image processing）利用这个特性把灰度图像变换为彩色图像，以增强人对图像细微差别的分辨力，广泛应用于X射线安检图像、医学图像、红外图像、遥感图像等领域。常用的伪彩色图像处理方法有灰度分层、灰度变换-合成等。

### 5.7.1 灰度分层
- 灰度分层是最简单的伪彩色图像处理方法，其基本思想是把图像的灰度级分成若干个区间，并为每个区间定义一种颜色，然后按照每个像素灰度值所属的区间，赋予该像素对应的颜色值。

### 示例：行李X射线安检图像的伪彩色处理

In [ ]:
#调用OpenCV、Pillow函数实现灰度分层方式的伪彩色处理
#读入一幅灰度图像
img_gray =io.imread('./imagedata/Xraysecuritycheck.jpg')

#采用OpenCV预定义颜色表
img_color1 = cv.applyColorMap(img_gray,cv.COLORMAP_JET)

#自定义颜色表，把256划分为4个颜色区间，依次赋值
usercolormap = np.zeros((1,256,3),dtype= np.uint8)
usercolormap[0,0:64,:] =    [255,0,0]   # red
usercolormap[0,64:128,:] =  [255,255,0] # yellow
usercolormap[0,128:192,:] = [0,255,0]   # green
usercolormap[0,192:256,:] = [0,0,255]   # blue

#OpenCV采用查表法转换为伪彩色图像
#先将灰度图像转换为具有3个颜色通道的消色图像
img_gray_3C = color.gray2rgb(img_gray)

#img_gray_3C = cv.cvtColor(img_gray, cv.COLOR_GRAY2RGB)

#查表得到RGB伪彩色图像
img_color2 = cv.LUT(img_gray_3C, usercolormap) 

#采用Numpy数组操作查表得到RGB伪彩色图像
#去掉颜色表第一维
usercmap = np.squeeze(usercolormap)
img_color3 = usercmap[img_gray]

#采用Matplotlib预定义颜色表
cmap = plt.get_cmap("copper")
img_Pseudocolor = (cmap(img_gray)*255).astype(np.uint8)


#调用Pillow函数,以索引图像格式实现伪彩色图像处理
#将灰度图像转换为PIL格式
img_ind = Image.fromarray(img_gray)
#将图像转换为索引图像
img_ind.convert('P')
#把自定义颜色表附加到索引图像中
img_ind.putpalette(list(usercmap.flatten()))
#保存索引图像
img_ind.save('Xraysecuritycheck_pc.png')

#显示结果
plt.figure(figsize=(19,19))
#输入灰度图像
plt.subplot(3,2,1); plt.imshow(img_gray ,vmin=0,vmax=255,cmap='gray')
plt.title('Input gray image') 
plt.axis('off')
#采用OpenCV预定义颜色表得到的伪彩色图像
plt.subplot(3,2,2); plt.imshow(img_color1[:,:,::-1])
plt.title('pesudocolor image using OpenCVs predefined colormap') 
plt.axis('off')
#采用自定义颜色表得到的RGB伪彩色图像
plt.subplot(3,2,3); plt.imshow(img_color2)
plt.title('pesudocolor RGB image using user colormap') 
plt.axis('off')
#采用自定义颜色表得到的索引图像
plt.subplot(3,2,4); plt.imshow(img_ind)
plt.title('pesudocolor indexed image using user colormap') 
plt.axis('off')

#采采用Matplotlib预定义颜色表得到的索引图像
plt.subplot(3,2,5); plt.imshow(img_Pseudocolor)
plt.title('pesudocolor RGB image using Matplotlib Predifined colormap') 
plt.axis('off')

plt.show()

### 5.7.2灰度变换-合成方法
- 灰度变换-合成方法的基本思想是将灰度图像每个像素的灰度值，通过三个不同的变换函数得到三个值，把这三个值看作是该像素颜色的R、G、B分量值，合成得到一幅RGB彩色图像。

### 示例：行李X射线安检图像的灰度变换-合成法伪彩色图像处理

In [ ]:
#灰度变换-合成法伪彩色图像处理

#读入一幅灰度图像
img_gray =io.imread('./imagedata/Xraysecuritycheck.jpg')

#将图像数据类型转换为[0,1]之间取值的浮点型
img_gray_f = util.img_as_float(img_gray)

#定义变换函数的控制参数
freq = 0.5  #Frequency
phase = 0.25 * np.pi  #Phase
#采用相位不同的正弦函数,计算RGB分量的变换值
img_r = np.abs(np.sin(2 * np.pi * freq * img_gray_f))
img_g = np.abs(np.sin(2 * np.pi * freq * img_gray_f + 0.25 * phase))
img_b = np.abs(np.sin(2 * np.pi * freq * img_gray_f + 0.5 * phase))

#把处理后的RGB三个颜色通道边缘组合起来
img_pesc = np.dstack((img_r,img_g,img_b))
#将数据类型转换为uint8
img_pesc = util.img_as_ubyte(img_pesc)

#显示结果
plt.figure(figsize=(16,8))
#输入灰度图像
plt.subplot(1,2,1); plt.imshow(img_gray ,vmin=0,vmax=255,cmap='gray')
plt.title('Input gray image') 
plt.axis('off')
#灰度变换-合成法得到的伪彩色图像
plt.subplot(1,2,2); plt.imshow(img_pesc)
plt.title('pesudocolor image') 
plt.axis('off')

plt.tight_layout()
plt.show()

# The end